In [ ]:
import pandas as pd
import sys
!{sys.executable} -m pip install pandas numpy pyarrow
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import d2_tweedie_score, mean_absolute_error


ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_freq = pq.read_table("clean_business_claims_freq.parquet").to_pandas()
df_sev = pq.read_table("clean_business_claims_sev.parquet").to_pandas()


In [ ]:
df_freq.sort_values(by="solar_system", inplace=True)
df_sev.sort_values(by="solar_system", inplace=True)

In [ ]:
#Grouping sev dataset based on policy id and summing claim_amount
agg_dict = {col: 'first' for col in df_sev.columns if col not in ['claim_id', 'claim_seq', 'policy_id', 'claim_amount']}
agg_dict['claim_amount'] = 'sum'

df_grouped_business_claims_sev = (
    df_sev.drop(columns=['claim_id', 'claim_seq'])
    .groupby('policy_id', as_index=False)
    .agg(agg_dict)
)

df_grouped_business_claims_sev.head(10)

In [ ]:
#merging the two datasets on policy_id
sev_claim_amounts = df_sev.groupby('policy_id', as_index=False)['claim_amount'].sum()

df_combined = df_freq.merge(
    sev_claim_amounts, 
    on='policy_id', 
    how='left'
)

df_combined['claim_amount'] = df_combined['claim_amount'].fillna(0)

df_combined.columns

df_combined.to_csv("bussiness_clams_model_data.csv", index=False)

NameError: name 'df_sev' is not defined